In [ ]:
import pickle

# File path
#file_path = "../Dataset/preprocessed_others_reviews_df.pkl"
file_path = "/content/drive/MyDrive/preprocessed_sw_reviews_df.pkl"   #da modificare se lancio in locale

with open(file_path, 'rb') as file:
  sw_reviews_df = pickle.load(file)
  #others_reviews_df = pickle.load(file)

print("Loaded dataset:", sw_reviews_df)

<ipython-input-2-e1935a5b2252>:8: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  sw_reviews_df = pickle.load(file)


Loaded dataset:      Review_ID   Movie_ID                                    Movie_Title  \
0      2221293  tt0076759             Star Wars: Episode IV - A New Hope   
1      4756672  tt0076759             Star Wars: Episode IV - A New Hope   
2      0156096  tt0076759             Star Wars: Episode IV - A New Hope   
3      0155657  tt0076759             Star Wars: Episode IV - A New Hope   
4      0155649  tt0076759             Star Wars: Episode IV - A New Hope   
...        ...        ...                                            ...   
7886   5552672  tt2527338  Star Wars: Episode IX - The Rise of Skywalker   
7887   5342141  tt2527338  Star Wars: Episode IX - The Rise of Skywalker   
7888   5334366  tt2527338  Star Wars: Episode IX - The Rise of Skywalker   
7889   5831082  tt2527338  Star Wars: Episode IX - The Rise of Skywalker   
7890   5339016  tt2527338  Star Wars: Episode IX - The Rise of Skywalker   

      Rating       Review_Date                                  Review_

In [ ]:
import subprocess
import sys

# List of required packages
required_packages = [
    "tf-keras", "numpy", "keybert"
]

def install_package(package):
    """Installs a package using pip if it's not already installed."""
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install missing packages
for package in required_packages:
    install_package(package)

Installing tf-keras...
numpy is already installed.
Installing keybert...


In [ ]:
from keybert import KeyBERT
from tqdm import tqdm

#specific_film = sw_reviews_df.loc[sw_reviews_df['Movie_ID'] == 'tt6751668']
kw_model = KeyBERT()
topics = dict()

#nell'altro caso uso "others_reviews_df"
for index, row in tqdm(sw_reviews_df.iterrows(), total=sw_reviews_df.shape[0]):
    specific_text = row['Processed_Review_Text']
    keywords = kw_model.extract_keywords(specific_text, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=5)
    topics[row['Review_ID']] = keywords

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 36192/36192 [49:39<00:00, 12.15it/s]


In [ ]:
import pandas as pd

# Creiamo una lista di dizionari con le parole chiave per ciascuna recensione
data = []
#for index, row in others_reviews_df.iterrows():
for index, row in sw_reviews_df.iterrows():
    review_id = row['Review_ID']
    movie_title = row['Movie_Title']  # Colonna contenente il titolo del film
    keywords = topics.get(review_id, [])  # Ottieni le parole chiave per la recensione

    # Aggiungiamo le parole chiave e i punteggi al dizionario
    data.append({
        "Review_ID": review_id,
        "Movie_Title": movie_title,
        "Keyword_1": keywords[0][0] if len(keywords) > 0 else "",
        "Score_1": keywords[0][1] if len(keywords) > 0 else 0,
        "Keyword_2": keywords[1][0] if len(keywords) > 1 else "",
        "Score_2": keywords[1][1] if len(keywords) > 1 else 0,
        "Keyword_3": keywords[2][0] if len(keywords) > 2 else "",
        "Score_3": keywords[2][1] if len(keywords) > 2 else 0,
        "Keyword_4": keywords[3][0] if len(keywords) > 3 else "",
        "Score_4": keywords[3][1] if len(keywords) > 3 else 0,
        "Keyword_5": keywords[4][0] if len(keywords) > 4 else "",
        "Score_5": keywords[4][1] if len(keywords) > 4 else 0,
    })

# Creiamo il DataFrame completo con le parole chiave
df_keywords = pd.DataFrame(data)

# Salviamo tutto in un unico file CSV
df_keywords.to_csv("all_keywords_results_sw.csv", index=False)
#df_keywords.to_csv("all_keywords_results_others.csv", index=False)

print("File CSV unico salvato con successo!")

# oppure se voglio salvare un file CSV per ciascun film:

#for movie_title, group in grouped:
#    file_name = f"{movie_title.replace(' ', '_')}_keywords.csv"  # Nome file senza spazi
#    group.to_csv(file_name, index=False)
#    print(f"File CSV per '{movie_title}' salvato con successo!")


File CSV unico salvato con successo!
